In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)
dataset=5 # 1-4:  whuGait,   5:  OU-ISIR

In [ ]:
data_path ='/content/drive/MyDrive/imuz/AutomaticExtractionData_IMUZCenter.rar'

In [ ]:

!apt-get install -y unrar
import os
rar_path = '/content/drive/MyDrive/imuz/AutomaticExtractionData_IMUZCenter.rar'
extract_path = '/content/drive/MyDrive/imuz/autoz/'
os.makedirs(extract_path, exist_ok=True)
!unrar x "{rar_path}" "{extract_path}"

In [ ]:
data_path ='/content/drive/MyDrive/imuz/autoz/AutomaticExtractionData_IMUZCenter/'

In [ ]:
import os
import csv
import numpy as np
import re
# Define the directory where your CSV files are located
csv_directory = data_path
mini=1000
for filename in os.listdir(csv_directory):
    if filename.endswith(".csv"):
        with open(os.path.join(csv_directory, filename), "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            # Skip the first two rows
            row=next(csv_reader)
            f=row[0][6:10]
            fint=int(f)
            if(fint<mini):
              mini=fint
print(mini)
K=int(mini/2)
print(K)


In [ ]:
K=128

In [ ]:
import os
import numpy as np
from csv import reader

csv_directory ='/content/drive/MyDrive/imuz/autoz/AutomaticExtractionData_IMUZCenter/'
array_list = []
array_list_ts = []
label = []
labelts = []

k = 128  

for filename in os.listdir(csv_directory):
    if filename.endswith(".csv"):
        person_id = filename.split("_")[1]  

        with open(os.path.join(csv_directory, filename), 'r') as csv_file:
            csv_reader = reader(csv_file)
            next(csv_reader) 
            data = [list(map(float, row[:6])) for row in csv_reader if len(row) >= 6]

        numlines = len(data)
        num_samples = numlines // k

        if num_samples < 5:
            splitsi = 1
        elif num_samples < 11:
            splitsi = 2
        elif num_samples < 16:
            splitsi = 3
        else:
            splitsi = 4


        segments = [data[i*k:(i+1)*k] for i in range(num_samples)]

        for i, segment in enumerate(segments):
            segment_np = np.array(segment)
            if i < splitsi:
                array_list_ts.append(segment_np)
                labelts.append(person_id)
            else:
                array_list.append(segment_np)
                label.append(person_id)

print(f"✅ number of training data  {len(array_list)}, number of testing data {len(array_list_ts)}")


In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling1D, Reshape, Dense, Multiply

def squeeze_excite_block(input_tensor):
    filters = input_tensor.shape[-1]
    se = GlobalAveragePooling1D()(input_tensor)
    se = Reshape((1, filters))(se)
    se = Dense(filters // 8, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)
    return Multiply()([input_tensor, se])


In [ ]:
from tensorflow.keras.layers import Conv1D, BatchNormalization, Activation, AveragePooling1D

def build_cnn_branch(input_tensor):
    x = Conv1D(24, 1, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling1D(pool_size=2)(x)
    x = squeeze_excite_block(x)

    x = Conv1D(40, 1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling1D(pool_size=2)(x)
    x = squeeze_excite_block(x)

    x = Conv1D(104, 1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling1D(pool_size=2)(x)
    return x


In [ ]:
from tensorflow.keras.layers import Masking, LSTM, Dropout

def build_lstm_branch(input_tensor):
    x = Masking()(input_tensor)
    x = LSTM(24, return_sequences=False)(x)
    x = Dropout(0.5)(x)
    return x



In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate, Dense, Reshape, Input, Flatten

input_layer = Input(shape=(128, 6))
cnn_output = build_cnn_branch(input_layer)
cnn_output = Flatten()(cnn_output)
lstm_output = build_lstm_branch(input_layer)

merged = Concatenate()([cnn_output, lstm_output])
final_output = Dense(num_classes, activation='softmax')(merged)

model = Model(inputs=input_layer, outputs=final_output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




In [ ]:
X_train = np.array(array_list)
X_test = np.array(array_list_ts)
y_train = np.array(label)
y_test = np.array(labelts)

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)


In [ ]:
from tensorflow.keras.utils import to_categorical

num_classes = len(np.unique(y_train_encoded))
y_train_cat = to_categorical(y_train_encoded, num_classes)
y_test_cat = to_categorical(y_test_encoded, num_classes)

In [ ]:
history = model.fit(
    X_train, y_train_cat,
    validation_data=(X_test, y_test_cat),
    epochs=50,
    batch_size=16
)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.title('Model Training Accuracy')
plt.show()


In [ ]:
import os
import csv
import numpy as np
import re
# Define the directory where your CSV files are located
csv_directory = data_path
# Initialize an array to store person IDs and arrays to store file data
person_ids = []
file_data = []
# Initialize an array to store the data from the current file
current_file_data = []
# Iterate through the files in the directory
rows = 1
columns = 6
# Create an empty 2D array
empty_2d_array = np.empty((rows, columns))
array_list = []
label=[]
for filename in os.listdir(csv_directory):
    if filename.endswith(".csv"):
        # Extract person ID from the filename
        person_id = filename.split("_")[1]
        # Store the person ID in the person_ids array
        person_ids.append(person_id)
        # Open and read the CSV file
        with open(os.path.join(csv_directory, filename), "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            # Skip the first two rows
            next(csv_reader)
            next(csv_reader)
            # Iterate through the remaining rows
            id=1
            row_array=[]
            empty_2d_array = np.empty((rows, columns))
            for row in csv_reader:
              # Convert each string to a NumPy array and append to the 'arrays' list
              arrays = [np.fromstring(string1, sep=",") for string1 in row]
              # Vertically stack the arrays to create a 2D NumPy array
              try:
                output_array = np.concatenate(arrays)
                empty_2d_array = np.vstack((empty_2d_array, output_array))
                id=id+1
              except:
                  id=1
                  empty_2d_array = np.empty((rows, columns))
              if(id%K==0):
                  array_list.append(empty_2d_array)
                  numbers_only = ''.join(re.findall(r'\d+', person_id))
                  label.append(int(numbers_only))
                  id=1
                  empty_2d_array = np.empty((rows, columns))
                # Add the person ID as a new element in the numpy array
                # Append the numpy array to the current_file_data array
first_shape = array_list[0].shape



In [ ]:
final = np.array(array_list)
print(final.shape)
print(len(label))

In [ ]:
X_train=final
# Find unique values and their numerical indices
unique_values, indices = np.unique(label, return_inverse=True)
# Replace the original values with their numerical indices
numerical_indices = indices.reshape(-1, 1)
# Create a NumPy array from the numerical indices
numerical_array = np.array(numerical_indices)
train_label=numerical_array
# Determine the number of bits required for binary representation
numerical_array = train_label
# Determine the maximum value in numerical_array
max_value = numerical_array.max()

# Initialize a new array filled with zeros
binary_representation = np.zeros((numerical_array.shape[0], max_value + 1), dtype=np.uint8)

# Use advanced indexing to set the corresponding elements to "1"
binary_representation[np.arange(numerical_array.shape[0]), numerical_array.ravel()] = 1

# Print the binary_representation
print(binary_representation.shape)

In [ ]:
train_label=binary_representation
X_train=final
print(train_label.shape)
print(X_train.shape)

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout
from sklearn.model_selection import train_test_split

# Assuming X_train, train_label, X_test, and test_label are already loaded and preprocessed

# Define the CNN + LSTM model
def create_model(input_shape, num_classes):
    model = Sequential()

    # CNN layers
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    # LSTM layer
    #model.add(LSTM(32, return_sequences=True))
    model.add(LSTM(512, return_sequences=False))
    # Dense layer
    model.add(Dropout(0.5))
    #model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    # Output layer
    model.add(Dense(num_classes, activation='sigmoid'))
    return model

# Set the input shape and number of classes
input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = train_label.shape[1]
# Create the model
model = create_model(input_shape, num_classes)
# Compile the model
model.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])
# Print the model summary
model.summary()

# Train the model
epochs = 300
batch_size = 32
X_train1, X_val, train_label1, val_label = train_test_split(X_train, train_label, test_size=0.15, random_state=42)
history = model.fit(X_train1, train_label1, epochs=epochs, batch_size=batch_size, validation_data=(X_val, val_label))


# Plot the accuracy and loss curves during training
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy / Loss')
plt.legend()
plt.title('Training and Validation Accuracy/Loss')
plt.show()


In [ ]:

# Plot the accuracy and loss curves during training
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy / Loss')
plt.legend()
plt.title('Training and Validation Accuracy/Loss')
plt.show()